# Model Editing

We use our IG and AP pipeline to localise important components. These components are edited using gradient descent to "unlearn" information. We evaluate our results on the CounterFact dataset.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import Tensor
import torch.optim as optim

from testing import logit_diff_metric
from applications.pipeline import run_attribution_steps, identify_target_components, optimise_edit_components, AttributionMethod, edit_model
from applications.datasets import CounterFact

from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
import copy

/vol/bitbucket/mwl21/fypvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
device = get_device()
# device = torch.device("cpu")
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

# Explicitly calculate and expose the result for each attention head
model.set_use_attn_result(True)
model.set_use_hook_mlp_in(True)

Loaded pretrained model gpt2-small into HookedTransformer


## Editing procedure

In [124]:
# Verify that loading works, for one example
n_samples = 5

counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=1)

clean_input, corrupted_input, labels = next(iter(counterfact_dataloader))
# clean_input, corrupted_input, labels = counterfact_dataset.get_single_sample(0)

# print(clean_input)
# print(corrupted_input)
# print(labels)


collate tensor([[[24111],
         [15823]]], device='cuda:0')


In [134]:
from applications.metrics import evaluate_counterfact_efficacy, evaluate_counterfact_paraphrased, evaluate_counterfact_neighborhood, evaluate_consistency
from applications.datasets import CounterFact
import pandas as pd
from collections import defaultdict

evaluation_scores = defaultdict(list)

for n, (clean_input, corrupted_input, labels) in enumerate(counterfact_dataloader):

    print("\nLABELS", labels.shape, "\n")

    original_output = model.generate(clean_input, max_new_tokens=5, do_sample=False)

    edited_models = edit_model(model, clean_input, corrupted_input, labels, n_epochs=5, overwrite=False)

    # Evaluate
    for i, edited_model in enumerate(edited_models):
        print(f"Prompt: {clean_input}")
        print("Original output:", original_output)
        # print(f"Original answer: {labels[:, 0]}. Target answer: {labels[:, 1]}")

        print(edited_model.generate(clean_input, max_new_tokens=5, do_sample=False))

        score, magnitude = evaluate_counterfact_efficacy(edited_model, n, verbose=True)
        evaluation_scores["Efficacy score"].append(score.item())
        evaluation_scores["Efficacy magnitude"].append(magnitude.item())

        score, magnitude = evaluate_counterfact_paraphrased(edited_model, n, verbose=False)
        evaluation_scores["Generalisation score"].append(score.item())
        evaluation_scores["Generalisation magnitude"].append(magnitude.item())

        score, magnitude = evaluate_counterfact_neighborhood(edited_model, n, verbose=False)
        evaluation_scores["Specificity score"].append(score.item())
        evaluation_scores["Specificity magnitude"].append(magnitude.item())

        consistency_score = evaluate_consistency(model, n, verbose=False)
        evaluation_scores["Consistency score"].append(score.item())
        evaluation_scores["Consistency magnitude"].append(magnitude.item())

    if n + 1 >= n_samples: break

evaluation_df = pd.DataFrame(evaluation_scores)
evaluation_df.to_csv('results/counterfact/evaluation.csv')

collate tensor([[[24111],
         [15823]]], device='cuda:0')

LABELS torch.Size([1, 2, 1]) 



100%|██████████| 5/5 [00:00<00:00, 58.07it/s]


Fine tuning model on sample 0...


torch.Size([1, 50257]) torch.Size([1, 2, 1])
Loss: 11.355487823486328
torch.Size([1, 50257]) torch.Size([1, 2, 1])
Loss: 8.931524276733398
torch.Size([1, 50257]) torch.Size([1, 2, 1])
Loss: 4.621245384216309
torch.Size([1, 50257]) torch.Size([1, 2, 1])
Loss: 1.2804584503173828
torch.Size([1, 50257]) torch.Size([1, 2, 1])
Loss: 0.062135644257068634
Prompt: ['The mother tongue of Danielle Darrieux is']
Original output: The mother tongue of Danielle Darrieux is French.

The


100%|██████████| 5/5 [00:00<00:00, 57.88it/s]

...


Fine tuning model on sample 0...
torch.Size([1, 50257]) torch.Size([1, 2, 3])
Loss: 16.295806884765625
torch.Size([1, 50257]) torch.Size([1, 2, 3])
Loss: 11.03549575805664
torch.Size([1, 50257]) torch.Size([1, 2, 3])
Loss: 6.588231086730957
torch.Size([1, 50257]) torch.Size([1, 2, 3])
Loss: 4.5115065574646
torch.Size([1, 50257]) torch.Size([1, 2, 3])
Loss: 3.4750988483428955
Prompt: ['What is the twin city of Lyon? It is']
Original output: What is the twin city of Lyon? It is a city of the French


100%|██████████| 5/5 [00:00<00:00, 55.49it/s]

What is the twin city of Lyon? It isBeauBeauBe


Original label: Beirut
Target label: Manila
Outputs: ['\n', '\n', ' their', ' their', ' their', ' their', '\n', ':', ':', ':']


100%|██████████| 5/5 [00:00<00:00, 48.75it/s]


## Evaluation

For each sample, we calculate the efficacy, generalisability, specificity and consistency for:

- The original models' outputs
- The edited model's outputs


In [75]:
evaluation_df.head()

,Efficacy score,Efficacy magnitude,Generalisation score,Generalisation magnitude,Specificity score,Specificity magnitude,Consistency score,Consistency magnitude
0,1.0,0.000148,1.0,0.000099,1.0,0.000164,1.0,0.000164
1,1.0,0.000020,1.0,0.000038,1.0,0.000367,1.0,0.000367
2,1.0,0.000063,1.0,0.000264,1.0,0.000456,1.0,0.000456
3,1.0,0.000573,0.5,-0.000062,1.0,0.000825,1.0,0.000825
4,0.3,0.000016,0.5,0.000071,0.6,0.003349,0.6,0.003349
